# Trying to evaluate a recipe given a Query

### Loading the search results and getting a sample recipe

In [41]:
import pandas as pd

output_path = 'output/Results_semantic.xlsx'

# Load the results
results = pd.read_excel(output_path, sheet_name="Sheet1")

# filtering the semantic questions
results = results[results['Tipo'] == 'Semantica']

# selecting a question
sample_result = results.iloc[0]

query, resulting_recipe = sample_result['Query'], sample_result['body']

print(f"Query: {query}")
print(f"Resulting Recipe heading: {resulting_recipe.split('\n')[0]}")

Query: I'm vegan. How can I make a bolognese?
Resulting Recipe heading: vegan bolognese


#### getting our gradding

In [58]:
our_reviews_paths = ['reviews/Results_semantic_Rodrigo.xlsx',
                     'reviews/Results_semantic_sanches.xlsx',
                     'reviews/Results_semantic_Vanessa.xlsx']

# Load the reviews
rod_reviews = pd.read_excel(our_reviews_paths[0], sheet_name="Sheet1")
san_reviews = pd.read_excel(our_reviews_paths[1], sheet_name="Sheet1")
van_reviews = pd.read_excel(our_reviews_paths[2], sheet_name="Sheet1")

# renaming cols
rod_reviews.rename(columns={'Unnamed: 12': 'Nota'}, inplace=True)
san_reviews.rename(columns={'nota': 'Nota'}, inplace=True)
van_reviews.rename(columns={'Unnamed: 6': 'Nota'}, inplace=True)

# concatenating the reviews and keeping only the relevant columns
reviews = pd.concat([rod_reviews, san_reviews, van_reviews])[['Query', 'body', 'Nota']]

# subtracting 1 from the ratings because its on 1-6 scale
reviews['Nota'] = reviews['Nota'] - 1

# filtering the reviews for the same question and recipe
reviews = reviews[(reviews['Query'] == query) & (reviews['body'] == resulting_recipe)]
reviews

,Query,body,Nota
25,I'm vegan. How can I make a bolognese?,vegan bolognese\n\nRecipe posted on: 2008-02-0...,5
25,I'm vegan. How can I make a bolognese?,vegan bolognese\n\nRecipe posted on: 2008-02-0...,5
25,I'm vegan. How can I make a bolognese?,vegan bolognese\n\nRecipe posted on: 2008-02-0...,5


### Making prompt templates

In [42]:


gpt_template = f"""
You are a query result validator assistant tasked with evaluating the quality of a given recipe in answering a given query.

Here is the recipe: \

{resulting_recipe}

Here is the query:\

{query}

Now evaluate from 0 to 5, the relevance of the recipe for answering the given query, where 0 is unrelated, 1 is poorly related, 2 is a little relevant but miss some important things, 3 is relevant but miss some restrictions, 4 is a relevant recipe that nearly matches all the possible criterias, and 5 is a perfect result, where every possible consideration and restrain included in the query is answered in the recipe. Include both your grading and a brief justificative of the grade.


Assistant grading:
Justificative:
"""

text_to_text_generation_template = f"""
You are a query result validator assistant tasked with evaluating the quality of a given recipe in answering a given query.

Here is the recipe: \

{resulting_recipe}

Here is the query:\

{query}

Now evaluate from 0 to 5, the relevance of the recipe for answering the given query, where 0 is unrelated, 1 is poorly related, 2 is a little relevant but miss some important things, 3 is relevant but miss some restrictions, 4 is a relevant recipe that nearly matches all the possible criterias, and 5 is a perfect result, where every possible consideration and restrain included in the query is answered in the recipe. Include both your grading and a brief justificative of the grade.

Evaluation and justification:
"""

### Loading a text-generation model

In [52]:
# text generation using IvanD2002/gemma-chatbot model
from transformers import pipeline

generator = pipeline('text-generation', model='IvanD2002/gemma-chatbot')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [43]:
res = generator(text_to_text_generation_template, max_new_tokens=500)

o modelo demorou 15m 11s para gerar a resposta...

### Verificando o texto gerado

In [45]:
print(res[0]['generated_text'])


You are a query result validator assistant tasked with evaluating the quality of a given recipe in answering a given query.

Here is the recipe: 
vegan bolognese

Recipe posted on: 2008-02-08

Tags: 30-minutes-or-less, time-to-make, course, main-ingredient, preparation, occasion, main-dish, pasta, dinner-party, vegetarian, dietary, comfort-food, pasta-rice-and-grains, spaghetti, taste-mood

Description: cooking vegan for my favorite meat eater is quite an enormous challenge for me sometimes.  i noticed that lauren had ordered spaghetti bolognese the last couple times we were in italian restaurants.  this was my vegan take on an old italian favorite, and she thought it was great! i decided to use textured vegetable protein (tvp) because it absorbs the flavors of whatever you rehydrate it with.

This recipe takes 30 minutes to be done.

For this recipe you will need the ingredients: 
olive oil
onion
carrot
garlic cloves
basil
oregano
thyme
bay leaf
crushed red pepper flakes
textured veg

In [50]:
given_grade, justificative = res[0]['generated_text'].split('Grade:')[1].split('Justification:')
given_grade, justificative = given_grade.strip(), justificative.strip()

print(f"Given grade: {given_grade}")
print(f"Justificative: {justificative}")

Given grade: 4
Justificative: This recipe is very relevant for answering the query, as it covers all the necessary steps and considerations for making a bolognese, including the use of textured vegetable protein, the preparation of the sauce, and the addition of the pasta.


### Testando com um modelo mais especifico e leve para question answering

In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/tinyroberta-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)


config.json:   0%|          | 0.00/835 [00:00<?, ?B/s]

C:\Users\lucas\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucas\.cache\huggingface\hub\models--deepset--tinyroberta-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/326M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [15]:
user_input = "I'm vegan. How can I make a bolognese?"
question = f"does the recipe include meat?"

context = resulting_recipe

QA_input = {
    'question': question,
    'context': context
}

res = nlp(QA_input)

res

{'score': 6.60974208699372e-08,
 'start': 263,
 'end': 276,
 'answer': 'cooking vegan'}

### checking the gpt prompt response

In [51]:
print(gpt_template)


You are a query result validator assistant tasked with evaluating the quality of a given recipe in answering a given query.

Here is the recipe: 
vegan bolognese

Recipe posted on: 2008-02-08

Tags: 30-minutes-or-less, time-to-make, course, main-ingredient, preparation, occasion, main-dish, pasta, dinner-party, vegetarian, dietary, comfort-food, pasta-rice-and-grains, spaghetti, taste-mood

Description: cooking vegan for my favorite meat eater is quite an enormous challenge for me sometimes.  i noticed that lauren had ordered spaghetti bolognese the last couple times we were in italian restaurants.  this was my vegan take on an old italian favorite, and she thought it was great! i decided to use textured vegetable protein (tvp) because it absorbs the flavors of whatever you rehydrate it with.

This recipe takes 30 minutes to be done.

For this recipe you will need the ingredients: 
olive oil
onion
carrot
garlic cloves
basil
oregano
thyme
bay leaf
crushed red pepper flakes
textured veg

#### Resposta no chatGpt:

Assistant grading: 5

Justificative: The recipe is a perfect vegan Bolognese, using TVP and vegetables, addressing the query fully with relevant ingredients and instructions.